In [1]:
import pandas as pd
from IPython.display import Markdown, HTML
from plotly import graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

from summer2 import AgeStratification, Overwrite
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
from summer2.functions.time import get_sigmoidal_interpolation_function

from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_birth_rate, get_pop_death_data, get_death_rates, get_population_entry_rate, get_death_adjs
from tb_incubator.input import add_latency_flow, display_summary

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

## Population data

In [2]:
# Load birth data
birth_rates = get_birth_rate()

In [3]:
# Load age-stratified, population and death data
pop_death = get_pop_death_data()
target_pops = pop_death.groupby(level=[0]).sum()["population"]

In [4]:
# Load death rates
death_rates = get_death_rates()

In [5]:
description = """This table provides snippets of the total population, total deaths, and birth rates from the dataset."""
markdown_table = display_summary(birth_rates, pop_death).to_markdown()
full_markdown = f"{description}\n\n{markdown_table}" # Combine description and table

# Display the full Markdown with description and table
Markdown(full_markdown)

This table provides snippets of the total population, total deaths, and birth rates from the dataset.

|      | Total Population   | Total Death   |   Birth rate |
|-----:|:-------------------|:--------------|-------------:|
| 1950 | 68,799,023         | 1,437         |       0.0405 |
| 1986 | 170,175,063        | 1,481         |       0.0282 |
| 2023 | 281,190,064        | 2,121         |       0.0159 |

## Model construction

In [6]:
# Arbitrary base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
model.set_initial_population({})

In [7]:
# TB transitions
model.add_death_flow("TB death", Parameter("death rate"), "infectious")

In [8]:
# Demographic transitions
model.add_universal_death_flows("population_death", Parameter("universal death"))  # Placeholder to overwrite later
model.add_replacement_birth_flow("replacement_birth", "susceptible")

In [9]:
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 100]]
age_strata = [i[0] for i in agegroup_request]
strat = AgeStratification("age", age_strata, model_comps)
death_adjs = get_death_adjs(death_rates, age_strata=age_strata)
strat.set_flow_adjustments("population_death", death_adjs)

# Apply age stratification with age-specific death rate functions of time
model.stratify_with(strat)

In [10]:
# Calculate population entry rates 
entry_rate = get_population_entry_rate(pop_death, 1850)

# Add births as additional entry rate (split imports in case the susceptible compartments are further stratified later)
model.add_importation_flow("births", entry_rate, dest="susceptible", split_imports=True, dest_strata={"age": "0"})

In [11]:
# Add latency flow
add_latency_flow(model)

In [12]:
params = {
    "death rate": 0.01,
    "universal death": 1.0,
    "stabilisation rate": 1.0,
    "early activation rate": 1.0,
    "late activation rate": 1.0,
}

In [13]:
# Track populations
age_pop_outputs = [model.request_output_for_compartments(s, model_comps, strata={"age": str(s)}) for s in age_strata]

In [14]:
Markdown(f"""
This model simulates the dynamics of the Indonesian population from **{int(model_times[0])}** to **{int(model_times[-1])}**. It will later be adjusted to model tuberculosis (TB) dynamics in Indonesia.

In this model, there will be:

- five compartments: **susceptible**, **early latent**, **late latent**, **infectious**, and **recovered**

- an intial population of 0

- age-stratification into five age groups: {', '.join(f"{group[0]}-{group[1]}" for group in agegroup_request)}

- inter-compartmental flows for deaths and births

""")


This model simulates the dynamics of the Indonesian population from **1850** to **2024**. It will later be adjusted to model tuberculosis (TB) dynamics in Indonesia.

In this model, there will be:

- five compartments: **susceptible**, **early latent**, **late latent**, **infectious**, and **recovered**

- an intial population of 0

- age-stratification into five age groups: 0-4, 5-14, 15-34, 35-49, 50-100

- inter-compartmental flows for deaths and births



## Results

In [15]:
# Run and inspect results
model.run(params)
fig = model.get_derived_outputs_df().plot.area()
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))